## 아래는 코드 샘플입니다. cluster_cate_weight_reflevel 작업은 일시 보류입니다.

In [1]:
# import modules
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps

In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
four_1 = pd.read_csv('./final/four/final_four_11.txt', sep='\t')
four_2 = pd.read_csv('./final/four/final_four_22.txt', sep='\t')
four_3 = pd.read_csv('./final/four/final_four_33.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-14 09:48:19.471280
종료: 2024-03-14 10:10:56.482976


In [4]:
four_1

,uid,cluster_id,pubyear,ref_n,refs_uid,cate_n,subject_traditional,within_core
0,WOS:000345609800009,30,2014,73,WOS:000205970500001,1,Law,0
1,WOS:000434794200003,30,2018,90,WOS:000205970500001,1,Law,0
2,WOS:000528191500001,30,2020,150,WOS:000205970500001,1,Law,0
3,WOS:000239862800001,911,2006,72,WOS:000200960000105,1,Biology,0
4,WOS:000653082100007,1351,2021,126,WOS:000188572100001,1,Geology,0
...,...,...,...,...,...,...,...,...
339005379,WOS:000270542600008,2367,2009,41,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005380,WOS:000257408200022,3252,2008,66,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005381,WOS:000286110900041,3308,2011,31,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005382,WOS:000329578200008,1749,2014,25,WOS:000170752800038,1,"Radiology, Nuclear Medicine & Medical Imaging",0


uid별 subject_category가 몇개인지 나타내주는 sub_n 컬럼을 생성합니다.

In [6]:
now = datetime.datetime.now() 
print('시작:', now)

four_1['sub_n'] = four_1.groupby('uid')['subject_traditional'].transform(lambda x: x.nunique())

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-14 11:20:11.220175
종료: 2024-03-14 12:07:28.803653


아래의 컬럼을 생성해줍니다.

## w1 = 1 / (sub_n)  
## w2 = 1 / (cate_n)  
## w = w1 * w2  
weight_cate : 한 클러스터 안에서 카테고리별(groupby) w를 합산(sum)한 값
norm_weight_cate : weight_cate를 unique한 uid 개수로 나눈 값
점검 : norm_weight_cate의 총계(sum)은 1이 되어야함.

아래는 코드 샘플입니다. cluster_cate_weight_reflevel 작업은 일시 보류입니다. 

In [7]:

now = datetime.datetime.now() 
print('시작:', now)


four_1['w1'] = 1 / four_1['sub_n']
four_1['w2'] = 1 / four_1['cate_n']


now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-14 12:07:28.808287
종료: 2024-03-14 12:07:30.639227


In [8]:
now = datetime.datetime.now() 
print('시작:', now)

four_2['w'] = four_1['w1'] * four_1['w2']


now = datetime.datetime.now() 
print('시작:', now)

시작: 2024-03-14 12:07:30.643090
시작: 2024-03-14 12:07:44.507512


In [ ]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)

four_ = pd.read_csv('./final/four.txt', sep='\t')


now = datetime.datetime.now()
print('종료:', now)

four(4번째 데이터셋의 최종파일)를 메모리 때문에 불러오지 못합니다.  
category embedding 작업은 cluster 단위로 aggregate하여 카테고리별 groupby를 해야합니다. 따라서 cluster_id 기준으로 분할하여 작업하는 방식이 있습니다. 
불러온 df들은('./final/four/final_four_11.txt', './final/four/final_four_22.txt', './final/four/final_four_33.txt') cluster_id 정렬이 안되어있습니다. 따라서 cluster_id 기준으로 분할 시에 추가 작업이 필요합니다. 
예를 들어, cluster_id 가 1~4xxx개 있는데, 4분할로 하여 3개의 df마다 쪼갭니다. 그 후 cluster_id 범위가 같은 파일을 병합합니다. -> weight계산합니다 -> 최종파일로 병합합니다.  